# Task 1

Consider the following tables: Products(ID, ProductName), Prices(ProductID REF Products(ID), Currency REF Rates(Currency), Price), Rates(Currency, PricePLN). Note that a product price may not be declared in all known currencies, but it always is declared in PLN as a reference. Prepare a batch script to update the Prices table based on Rates table. In case there is a row in Prices which references a currency that no longer exists in Rates, the row should be removed. This task should be implemented using cursors.

In [17]:
DROP TABLE IF EXISTS dbo.Prices
DROP TABLE IF EXISTS dbo.Products
DROP TABLE IF EXISTS dbo.Rates

CREATE TABLE dbo.Products (
    ID INT IDENTITY, 
    ProductName VARCHAR(200),
    CONSTRAINT Products_PK PRIMARY KEY (ID)
);

CREATE TABLE dbo.Rates (
    Currency VARCHAR(3),
    PricePLN FLOAT,
    CONSTRAINT currency_unique UNIQUE (Currency)
);

CREATE TABLE dbo.Prices (
    ProductID INT,
    Currency VARCHAR(3),
    Price FLOAT,
    CONSTRAINT currency_ProductID_unique_together UNIQUE (ProductID, Currency),
    CONSTRAINT ProductID_FK FOREIGN KEY (ProductID) REFERENCES dbo.Products (ID) ON DELETE CASCADE,
    CONSTRAINT Currency_FK FOREIGN KEY (Currency) REFERENCES dbo.Rates (Currency) ON DELETE CASCADE
);

SET IDENTITY_INSERT dbo.Products ON
INSERT INTO dbo.Products (ID, ProductName) VALUES
(1, 'car'),
(2, 'book'),
(3, 'window'),
(4, 'vodka'),
(5, 'snow'),
(6, 'mushrooms')
SET IDENTITY_INSERT dbo.Products OFF

INSERT INTO dbo.Rates (Currency, PricePLN) VALUES
('EUR', 4.20),
('USD', 4.0),
('PLN', 1.0),
('GBP', 5),
('RUB', 0.05)

INSERT INTO dbo.Prices (ProductID, Currency, Price) VALUES
(1, 'USD', 32000.0),
(1, 'PLN', 130000.0),
(1, 'EUR', 30000.0),
(1, 'GBP', 22000.0),
(2, 'PLN', 100.0),
(2, 'EUR', 22.0),
(2, 'USD', 26.0),
(3, 'RUB', 100000.0),
(3, 'PLN', 1000.0),
(4, 'PLN', 60.0),
(4, 'USD', 30.0),
(5, 'USD', 40.0),
(5, 'PLN', 350.0),
(5, 'EUR', 38.0),
(5, 'GBP', 35.0),
(6, 'USD', 20.0),
(6, 'PLN', 100.0),
(6, 'EUR', 18.0),
(6, 'GBP', 15.0)

SELECT * FROM dbo.Products
SELECT * FROM dbo.Rates
SELECT * FROM dbo.Prices

(6 rows affected)

(5 rows affected)

(19 rows affected)

(6 rows affected)

(5 rows affected)

(19 rows affected)

Total execution time: 00:00:00.139

ID,ProductName
1,car
2,book
3,window
4,vodka
5,snow
6,mushrooms


Currency,PricePLN
EUR,4.2
USD,4
PLN,1
GBP,5
RUB,0.05


ProductID,Currency,Price
1,USD,32000
1,PLN,130000
1,EUR,30000
1,GBP,22000
2,PLN,100
2,EUR,22
2,USD,26
3,RUB,100000
3,PLN,1000
4,PLN,60


In [12]:
DECLARE prices_update CURSOR FOR SELECT ProductID as id, Currency as c, Price as p FROM dbo.Prices ORDER BY id
DECLARE @id INT, @old_id INT = -1, @currency VARCHAR(3), @price FLOAT, @PLN_price FLOAT, @curr_price FLOAT

OPEN prices_update
FETCH NEXT from prices_update into @id, @currency, @price
WHILE (@@FETCH_STATUS = 0)
BEGIN
    IF @currency != 'PLN'
        IF @id != @old_id
            SET @old_id = @id
            SELECT @PLN_price = Price FROM dbo.Prices WHERE Currency = 'PLN' AND ProductID = @id

        SELECT @curr_price = PricePLN FROM dbo.Rates WHERE Currency = @currency
        UPDATE dbo.Prices SET Price = @PLN_price / @curr_price WHERE CURRENT OF prices_update

    FETCH NEXT from prices_update into @id, @currency, @price
END
CLOSE prices_update
DEALLOCATE prices_update

SELECT * FROM dbo.Prices

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

(1 row affected)

(19 rows affected)

Total execution time: 00:00:00.189

ProductID,Currency,Price
1,USD,32500
1,PLN,130000
1,EUR,30952.38095238095
1,GBP,26000
2,PLN,100
2,EUR,23.80952380952381
2,USD,25
3,RUB,20000
3,PLN,1000
4,PLN,60


# Task 3
Consider the following tables: Cache(ID, UrlAddress, LastAccess), History(ID, UrlAddress, LastAccess) and Parameters(Name, Value). The meaning of fields of Cache and History tables is as follows:
• ID — row identifier,
• UrlAddress — the address of a website, 
• LastAccess — the time of the last visit with with an accuracy of a second.
The Parameters table contains only one row with Name = max cache and Value set to the maximum size of the Cache table. At the time of inserting of a new row to Cache table the following conditions should be met:
• If the cache already contains the address of the website being inserted, the last access time should be only modified,
 • Otherwise, the size of the cache should be compared to the value in the max cache parameter from the Parameters table. There are two cases: 
> – If it’s lower, the new row should be inserted.
> 
> – Otherwise, determine a website for which the last access is the oldest (if there are more than one, pick any of them). The corresponding row should be transferred to the History table, but in case it refers to a website already present there, the last access time should be only modified.
This task should be implemented using triggers.

In [4]:
DROP TABLE IF EXISTS dbo.Cache
DROP TABLE IF EXISTS dbo.History
DROP TABLE IF EXISTS dbo.Parameters

CREATE TABLE dbo.Cache (
    ID INT IDENTITY, 
    UrlAddress VARCHAR(200),
    LastAccess DATETIME,
    CONSTRAINT Cache_ID_PK PRIMARY KEY (ID),
    CONSTRAINT Cache_UrlAddress_unique UNIQUE (UrlAddress)
);

CREATE TABLE dbo.History (
    ID INT IDENTITY, 
    UrlAddress VARCHAR(200),
    LastAccess DATETIME,
    CONSTRAINT History_ID_PK PRIMARY KEY (ID),
    CONSTRAINT History_UrlAddress_unique UNIQUE (UrlAddress)
);

CREATE TABLE dbo.Parameters (
    Name VARCHAR(20),
    Value INT,
    CONSTRAINT Parameters_Name_unique UNIQUE (Name)
);

INSERT INTO dbo.Parameters (Name, Value) VALUES
('max_cache', 4)

INSERT INTO dbo.Cache (UrlAddress, LastAccess) VALUES
('https://github.com/', CONVERT(datetime, '16/11/2012 04:00:00', 103)),
('https://google.com/', CONVERT(datetime, '16/11/2013 00:01:00', 103)),
('https://facebook.com/', CONVERT(datetime, '16/11/2014 00:02:00', 103)),
('https://amazon.com/', CONVERT(datetime, '16/11/2013 03:03:00', 103))

INSERT INTO dbo.History (UrlAddress, LastAccess) VALUES
('https://github.com/', CONVERT(datetime, '16/11/2012 00:00:00', 103)),
('https://google.com/', CONVERT(datetime, '16/11/2012 00:00:00', 103)),
('https://facebook.com/', CONVERT(datetime, '16/11/2012 00:00:00', 103)),
('https://amazon.com/', CONVERT(datetime, '16/11/2012 00:00:00', 103)),
('https://ii.uwr.pl/', CONVERT(datetime, '16/11/2012 00:00:00', 103)),
('https://allegro.pl/', CONVERT(datetime, '16/11/2012 00:00:00', 103)),
('https://godaddy.pl/', CONVERT(datetime, '16/11/2012 00:00:00', 103)),
('https://home.pl/', CONVERT(datetime, '16/11/2012 00:00:00', 103))

SELECT * FROM dbo.Parameters
SELECT * FROM dbo.Cache
SELECT * FROM dbo.History

(1 row affected)

(4 rows affected)

(8 rows affected)

(1 row affected)

(4 rows affected)

(8 rows affected)

Total execution time: 00:00:00.119

Name,Value
max_cache,4


ID,UrlAddress,LastAccess
1,https://github.com/,2012-11-16 04:00:00.000
2,https://google.com/,2013-11-16 00:01:00.000
3,https://facebook.com/,2014-11-16 00:02:00.000
4,https://amazon.com/,2013-11-16 03:03:00.000


ID,UrlAddress,LastAccess
1,https://github.com/,2012-11-16 00:00:00.000
2,https://google.com/,2012-11-16 00:00:00.000
3,https://facebook.com/,2012-11-16 00:00:00.000
4,https://amazon.com/,2012-11-16 00:00:00.000
5,https://ii.uwr.pl/,2012-11-16 00:00:00.000
6,https://allegro.pl/,2012-11-16 00:00:00.000
7,https://godaddy.pl/,2012-11-16 00:00:00.000
8,https://home.pl/,2012-11-16 00:00:00.000


In [22]:
DROP trigger IF EXISTS cache_or_history

Commands completed successfully.

Total execution time: 00:00:00.036

In [24]:
CREATE TRIGGER cache_or_history
ON dbo.Cache INSTEAD OF INSERT
AS
BEGIN
    DECLARE distribute_inserted_data CURSOR FOR SELECT UrlAddress, LastAccess FROM INSERTED
    DECLARE @UrlAddress VARCHAR(200), @LastAccess DATETIME,
            @rows_no INT, @max_rows_no INT,
            @url_cnt INT = 0,
            @OldestUrlAddress VARCHAR(200), @OldestLastAccess DATETIME

    SELECT TOP 1 @max_rows_no = Value FROM dbo.Parameters;

    OPEN distribute_inserted_data
    FETCH NEXT from distribute_inserted_data into @UrlAddress, @LastAccess
    WHILE (@@FETCH_STATUS = 0)
    BEGIN
        SET @url_cnt = 0
        SELECT @url_cnt = COUNT(ID) FROM dbo.Cache WHERE UrlAddress = @UrlAddress
        IF @url_cnt = 1
        BEGIN
            PRINT 'This url already is in cache so just updating it.'
            UPDATE dbo.Cache SET LastAccess = @LastAccess WHERE UrlAddress = @UrlAddress
        END
        ELSE
        BEGIN
            SELECT @rows_no = COUNT(*) FROM dbo.Cache
            IF @rows_no < @max_rows_no
            BEGIN
                PRINT 'This url is not in cache so adding it.'
                INSERT INTO dbo.Cache VALUES (@UrlAddress, @LastAccess)
            END
            ELSE
            BEGIN
                SELECT TOP 1 @OldestUrlAddress = UrlAddress, @OldestLastAccess = LastAccess FROM dbo.Cache ORDER BY LastAccess DESC
                DELETE FROM dbo.Cache WHERE UrlAddress = @OldestUrlAddress

                SET @url_cnt = 0
                SELECT @url_cnt = COUNT(ID) FROM dbo.History WHERE UrlAddress = @OldestUrlAddress
                IF @url_cnt = 1
                BEGIN
                    PRINT 'Cache is full lets update url in the history'
                    UPDATE dbo.History SET LastAccess = @OldestLastAccess WHERE UrlAddress = @OldestUrlAddress
                    INSERT INTO dbo.Cache VALUES (@UrlAddress, @LastAccess)
                END
                ELSE
                BEGIN
                    PRINT 'Cache is full lets add new to the history'
                    INSERT INTO dbo.History VALUES (@OldestUrlAddress, @OldestLastAccess)
                    INSERT INTO dbo.Cache VALUES (@UrlAddress, @LastAccess)
                END
            END
        END
        FETCH NEXT from distribute_inserted_data into @UrlAddress, @LastAccess
    END
    CLOSE distribute_inserted_data
    DEALLOCATE distribute_inserted_data
END

Commands completed successfully.

Total execution time: 00:00:00.043

In [12]:
SELECT * FROM dbo.Cache
SELECT * FROM dbo.History
SELECT * FROM dbo.Parameters

(4 rows affected)

(8 rows affected)

(1 row affected)

Total execution time: 00:00:00.050

ID,UrlAddress,LastAccess
1,https://github.com/,2012-11-16 04:00:00.000
2,https://google.com/,2013-11-16 00:01:00.000
3,https://facebook.com/,2014-11-16 00:02:00.000
4,https://amazon.com/,2013-11-16 03:03:00.000


ID,UrlAddress,LastAccess
1,https://github.com/,2012-11-16 00:00:00.000
2,https://google.com/,2012-11-16 00:00:00.000
3,https://facebook.com/,2012-11-16 00:00:00.000
4,https://amazon.com/,2012-11-16 00:00:00.000
5,https://ii.uwr.pl/,2012-11-16 00:00:00.000
6,https://allegro.pl/,2012-11-16 00:00:00.000
7,https://godaddy.pl/,2012-11-16 00:00:00.000
8,https://home.pl/,2012-11-16 00:00:00.000


Name,Value
max_cache,4


In [25]:
INSERT INTO dbo.Cache VALUES ('https://github.com/', CONVERT(datetime, '16/11/2012 00:22:22', 103))
SELECT * FROM dbo.Cache
SELECT * FROM dbo.History

This url already is in cache so just updating it.

(1 row affected)

(1 row affected)

(4 rows affected)

(8 rows affected)

Total execution time: 00:00:00.074

ID,UrlAddress,LastAccess
1,https://github.com/,2012-11-16 00:22:22.000
2,https://google.com/,2013-11-16 00:01:00.000
3,https://facebook.com/,2014-11-16 00:02:00.000
4,https://amazon.com/,2013-11-16 03:03:00.000


ID,UrlAddress,LastAccess
1,https://github.com/,2012-11-16 00:00:00.000
2,https://google.com/,2012-11-16 00:00:00.000
3,https://facebook.com/,2012-11-16 00:00:00.000
4,https://amazon.com/,2012-11-16 00:00:00.000
5,https://ii.uwr.pl/,2012-11-16 00:00:00.000
6,https://allegro.pl/,2012-11-16 00:00:00.000
7,https://godaddy.pl/,2012-11-16 00:00:00.000
8,https://home.pl/,2012-11-16 00:00:00.000


In [ ]:
INSERT INTO dbo.Cache VALUES ('https://some.pl/', CONVERT(datetime, '16/11/2015 00:40:22', 103))
SELECT * FROM dbo.Cache
SELECT * FROM dbo.History

(1 row affected)

https://facebook.com/

1

Cache is full lets update url in the history

(1 row affected)

...

(4 rows affected)

(8 rows affected)

Total execution time: 00:00:00.058

ID,UrlAddress,LastAccess
1,https://github.com/,2012-11-16 00:22:22.000
2,https://google.com/,2013-11-16 00:01:00.000
4,https://amazon.com/,2013-11-16 03:03:00.000
5,https://some.pl/,2015-11-16 00:40:22.000


ID,UrlAddress,LastAccess
1,https://github.com/,2012-11-16 00:00:00.000
2,https://google.com/,2012-11-16 00:00:00.000
3,https://facebook.com/,2014-11-16 00:02:00.000
4,https://amazon.com/,2012-11-16 00:00:00.000
5,https://ii.uwr.pl/,2012-11-16 00:00:00.000
6,https://allegro.pl/,2012-11-16 00:00:00.000
7,https://godaddy.pl/,2012-11-16 00:00:00.000
8,https://home.pl/,2012-11-16 00:00:00.000
